'Automatically' retires brownfield land endpoints. 

Needs a file `duplicates_list_file` per the code below, containing these fields: "resource", "endpoint", "ep_start_date", "ACTION", where the ACTION colun contains the letter R meaning Retire this one.

it reads the duplicates file and creates two files endpoint_new.csv and old-resource_new.csv

1. Runs through the above file, locating endpoints and setting their end-date to the dead-date (below).
1. Adds the resource to the old-resource file, setting their status to 410, meaning 'gone away'.


In [ ]:
duplicates_list_file = r'/mnt/c/Users/MarkSmith/Downloads/Brownfield_Duplicates.csv'

source_folder= "/home/mark/DLUHC/brownfield-land-collection/collection/"

dead_date = "2023-07-24"


In [ ]:
import pandas as pd
import numpy as np


In [ ]:

all_duplicates = pd.read_csv(duplicates_list_file)

all_duplicates.head()

In [ ]:
# Load the files that make up the collection

old_resource_file = F"{source_folder}/old-resource.csv"
old_resource = pd.read_csv(old_resource_file)

endpoint_file =  F"{source_folder}/endpoint.csv"
endpoints = pd.read_csv(endpoint_file)

source_file = F"{source_folder}/source.csv"
source =  pd.read_csv(source_file)

In [ ]:

duplicates = all_duplicates [all_duplicates["ACTION"] == "R"]
duplicates = duplicates [duplicates["end_date"] != ""]

duplicates = duplicates[["resource", "endpoint", "ep_start_date", "end_date", "ACTION"]]



len(duplicates)
#duplicatea.head()

In [ ]:
def subDate (current):
    if pd.isnull(current):
        return dead_date
    else: 
        return current

for duplicate_row in duplicates.iterrows():
    resource_value = duplicate_row[1].values[0]
    endpoint_value = duplicate_row[1].values[1]
    start_date_value = duplicate_row[1].values[2]
    end_date_value = duplicate_row[1].values[3]
    
    sub = subDate (end_date_value)
    
    print (F"{resource_value} // {endpoint_value} // {start_date_value} // {end_date_value} // {sub} ")

    # end it in endpint.csv
    endpoints.loc[endpoints.endpoint == endpoint_value, 'end-date'] = sub
    
    # write it to the old_resource file'
    old_resource.loc[len(old_resource)] = {'old-resource': resource_value, 'status': 410, 'resource': ''}


endpoint_file_out =  F"{source_folder}/endpoint_new.csv"
endpoints.to_csv (endpoint_file_out, index=False)

old_resource_file_out = F"{source_folder}/old-resource_new.csv"
old_resource.to_csv(old_resource_file_out, index=False)